In [2]:
# MILP_SOS.ipynb
#
# Objetivo: Resolver o problema v2.3 usando a abordagem de
#           linearização clássica: Piecewise-Linear (PWL)
#           com restrições SOS2.
#
import gurobipy as gp
from gurobipy import GRB
import time
import numpy as np

# --- 1. Definição dos Dados do Problema (v2.3) ---
# (Exatamente o mesmo que o notebook MINLP anterior)

# 48 períodos de 30 minutos
T = range(1, 49)

# Demanda por período (MW)
D_lista = [
    90, 98, 95, 93, 90, 90, 92, 95, 100, 105, 110, 120, 130, 140, 150, 165, 
    180, 195, 210, 225, 240, 245, 250, 248, 245, 240, 230, 225, 220, 215, 
    210, 205, 200, 195, 190, 200, 210, 220, 230, 240, 235, 230, 215, 200, 
    175, 150, 130, 115
]
D = {t: D_lista[t-1] for t in T}

# --- Usina a Gás (Linear) ---
gas_params = {
    'var_cost': 80,    # R$/MW
    'fix_cost': 250,   # R$/período
    'max_cap': 150,    # MW
    'min_gen': 45,     # MW
    'ramp_rate': 50,   # MW/30min
    'min_uptime': 3    # períodos
}

# --- Usina a Carvão (Não-Linear) ---
carvao_params = {
    # Custo Variável: 60*P + 0.2*P^2
    'var_cost_linear': 60,    # R$/MW
    'var_cost_quad': 0.2,     # R$/MW^2
    'fix_cost': 1000,         # R$/período
    'max_cap': 200,           # MW
    'min_gen': 30,            # MW
    'ramp_rate': 40,          # MW/30min
    'min_uptime': 1           # períodos
}

# --- 2. NOVA ETAPA: Definição da Linearização PWL-SOS2 ---

# Esta é a função não-linear que queremos aproximar
def f_custo_carvao(P):
    if P == 0:
        return 0.0
    # Custo = 60*P + 0.2*P^2
    return (carvao_params['var_cost_linear'] * P +
            carvao_params['var_cost_quad'] * P * P)

# Vamos criar os breakpoints para a linearização.
# O domínio da função quando a usina está LIGADA é [30, 200].
# Vamos usar 10 segmentos (11 pontos) para cobrir este domínio.
n_segments = 10
P_points_active = np.linspace(
    carvao_params['min_gen'],  # 30
    carvao_params['max_cap'],  # 200
    n_segments + 1
)

# Precisamos também do ponto (0, 0) para quando a usina está DESLIGADA.
# Nossos pontos de Potência (P)
P_points = [0.0] + list(P_points_active)
# Nossos pontos de Custo (C)
Cost_points = [f_custo_carvao(p) for p in P_points]
n_points = len(P_points)

print("--- Configuração da Linearização PWL ---")
print(f"Pontos de Potência (P): {[round(p, 1) for p in P_points]}")
print(f"Pontos de Custo (C): {[round(c, 1) for c in Cost_points]}")
print("-" * 37)

# --- 3. Criação do Modelo ---
m = gp.Model("MILP_Despacho_SOS2")

# --- 4. Definição das Variáveis de Decisão ---
T_with_0 = range(0, 49) 
P_gas = m.addVars(T_with_0, name="P_gas", lb=0.0)
P_carvao = m.addVars(T_with_0, name="P_carvao", lb=0.0)
z_gas = m.addVars(T_with_0, name="z_gas", vtype=GRB.BINARY)
z_carvao = m.addVars(T_with_0, name="z_carvao", vtype=GRB.BINARY)
start_gas = m.addVars(T, name="start_gas", vtype=GRB.BINARY)
start_carvao = m.addVars(T, name="start_carvao", vtype=GRB.BINARY)

# --- NOVAS Variáveis para a Linearização ---
# C_var_carvao_hat: O custo variável *aproximado* da usina a carvão
C_var_carvao_hat = m.addVars(T, name="C_var_carvao_hat", lb=0.0)

# lambdas: Os pesos da combinação convexa para a linearização
# Teremos 'n_points' (12) pesos para cada um dos 48 períodos
lambdas = m.addVars(T, range(n_points), name="lambda", lb=0.0)


# --- 5. Definição da Função Objetivo (AGORA LINEAR) ---
custo_gas = gp.quicksum(
    gas_params['var_cost'] * P_gas[t] + gas_params['fix_cost'] * z_gas[t]
    for t in T
)

# Substituímos a fórmula quadrática pela nossa variável de custo aproximado
custo_carvao_linearizado = gp.quicksum(
    C_var_carvao_hat[t] + carvao_params['fix_cost'] * z_carvao[t]
    for t in T
)

m.setObjective(custo_gas + custo_carvao_linearizado, GRB.MINIMIZE)

# --- 6. Definição das Restrições ---

# Estado Inicial (t=0)
m.addConstr(P_gas[0] == 0)
m.addConstr(z_gas[0] == 0)
m.addConstr(P_carvao[0] == 0)
m.addConstr(z_carvao[0] == 0)

# Restrições para cada período t = 1 a 48
for t in T:
    
    # --- Restrições do Problema Original (idênticas ao MINLP) ---
    m.addConstr(P_gas[t] + P_carvao[t] >= D[t], name=f"Demanda_t{t}")
    m.addConstr(P_gas[t] <= gas_params['max_cap'] * z_gas[t], name=f"Gas_MaxCap_t{t}")
    m.addConstr(P_gas[t] >= gas_params['min_gen'] * z_gas[t], name=f"Gas_MinGen_t{t}")
    m.addConstr(P_carvao[t] <= carvao_params['max_cap'] * z_carvao[t], name=f"Carvao_MaxCap_t{t}")
    m.addConstr(P_carvao[t] >= carvao_params['min_gen'] * z_carvao[t], name=f"Carvao_MinGen_t{t}")
    m.addConstr(P_gas[t] - P_gas[t-1] <= gas_params['ramp_rate'], name=f"Gas_RampUp_t{t}")
    m.addConstr(P_gas[t-1] - P_gas[t] <= gas_params['ramp_rate'], name=f"Gas_RampDown_t{t}")
    m.addConstr(P_carvao[t] - P_carvao[t-1] <= carvao_params['ramp_rate'], name=f"Carvao_RampUp_t{t}")
    m.addConstr(P_carvao[t-1] - P_carvao[t] <= carvao_params['ramp_rate'], name=f"Carvao_RampDown_t{t}")
    m.addConstr(start_gas[t] >= z_gas[t] - z_gas[t-1], name=f"Gas_StartDef_t{t}")
    m.addConstr(start_carvao[t] >= z_carvao[t] - z_carvao[t-1], name=f"Carvao_StartDef_t{t}")
    
    k_gas = gas_params['min_uptime']
    for tau in range(t, min(t + k_gas, 49)): 
        m.addConstr(z_gas[tau] >= start_gas[t], name=f"Gas_MinUptime_t{t}_tau{tau}")

    k_carvao = carvao_params['min_uptime']
    for tau in range(t, min(t + k_carvao, 49)):
        m.addConstr(z_carvao[tau] >= start_carvao[t], name=f"Carvao_MinUptime_t{t}_tau{tau}")

    # --- NOVAS Restrições da Linearização PWL-SOS2 ---
    
    # 1. A soma dos pesos (lambdas) deve ser 1 se a usina está LIGADA (z=1)
    #    e 0 se está DESLIGADA (z=0).
    m.addConstr(
        lambdas.sum(t, '*') == z_carvao[t], 
        name=f"Lambda_Sum_t{t}"
    )

    # 2. Reconstrução da Potência (P):
    #    P_carvao[t] deve ser a combinação convexa dos P_points
    m.addConstr(
        P_carvao[t] == gp.quicksum(P_points[k] * lambdas[t, k] for k in range(n_points)),
        name=f"P_Recon_t{t}"
    )

    # 3. Reconstrução do Custo (C_hat):
    #    C_var_carvao_hat[t] deve ser a MESMA combinação convexa dos Cost_points
    m.addConstr(
        C_var_carvao_hat[t] == gp.quicksum(Cost_points[k] * lambdas[t, k] for k in range(n_points)),
        name=f"Cost_Recon_t{t}"
    )

    # 4. Restrição SOS2:
    #    Diz ao Gurobi que no máximo 2 'lambdas' podem ser não-zeros,
    #    e eles devem ser adjacentes. Isso garante a linearização correta.
    m.addSOS(GRB.SOS_TYPE2, [lambdas[t, k] for k in range(n_points)])


# --- 7. Otimização do Modelo ---
print("\n--- Iniciando Otimização do Modelo MILP-SOS2 ---")
m.setParam('Presolve', 0)
start_time = time.time()

m.optimize()
end_time = time.time()
print("--- Otimização Concluída ---")

solve_time = end_time - start_time

# --- 8. Exibição dos Resultados ---
if m.Status == GRB.OPTIMAL:
    print(f"\nSolução Ótima Encontrada!")
    # Nota: O custo será ligeiramente DIFERENTE do MINLP,
    # pois esta é uma APROXIMAÇÃO.
    print(f"Custo Total (MILP-SOS2): R$ {m.ObjVal:.2f}")
    print(f"Tempo de Solução: {solve_time:.4f} segundos")
    print("-" * 70)
    
    print("Resumo da Operação (Geração em MW):")
    print("Per. | Demanda | Gás (P)   | Gás (z) | Carvão (P) | Carvão (z) | Total Gen. | Sobra")
    print("-" * 70)
    for t in T:
        gas_gen = P_gas[t].X
        gas_z = "ON" if z_gas[t].X > 0.5 else "OFF"
        carvao_gen = P_carvao[t].X
        carvao_z = "ON" if z_carvao[t].X > 0.5 else "OFF"
        total_gen = gas_gen + carvao_gen
        sobra = total_gen - D[t]
        
        print(f" {t:2d} |  {D[t]:3d}  | {gas_gen:8.2f} | {gas_z:^7} | {carvao_gen:10.2f} | {carvao_z:^10} | {total_gen:10.2f} | {sobra:5.2f}")

elif m.Status == GRB.INFEASIBLE:
    print("\n--- ERRO: Modelo Inviável ---")
    m.computeIIS()
    m.write("modelo_conflito.ilp")
    print("IIS salvo em 'modelo_conflito.ilp'.")
else:
    print(f"Otimização terminou com status: {m.Status}")

--- Configuração da Linearização PWL ---
Pontos de Potência (P): [0.0, np.float64(30.0), np.float64(47.0), np.float64(64.0), np.float64(81.0), np.float64(98.0), np.float64(115.0), np.float64(132.0), np.float64(149.0), np.float64(166.0), np.float64(183.0), np.float64(200.0)]
Pontos de Custo (C): [0.0, np.float64(1980.0), np.float64(3261.8), np.float64(4659.2), np.float64(6172.2), np.float64(7800.8), np.float64(9545.0), np.float64(11404.8), np.float64(13380.2), np.float64(15471.2), np.float64(17677.8), np.float64(20000.0)]
-------------------------------------

--- Iniciando Otimização do Modelo MILP-SOS2 ---
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
Presolve  0

Optimize a model with 865 rows, 916 columns and 3310 nonzeros
Model 